# Batch Gradient Descent and Stochastic Gradient Descent

We will use very simple home prices data set to implement mini batch gradient descent in python.
1. Batch gradient descent uses all training samples in forward pass to calculate cumulitive error and than we adjust weights using derivaties

2. Stochastic GD: we randomly pick one training sample, perform forward pass, compute the error and immidiately adjust weights

3. Mini batch GD: we use a batch of m samples where 0 < m < n (where n is total number of training samples)

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('homeprices_banglore.csv')
df.sample(5)

,area,bedrooms,price
17,1540,3,60.0
5,1170,2,38.0
8,1310,3,50.0
12,1000,2,38.0
15,1175,2,42.0


Preprocessing/Scaling: Since our columns are on different sacle it is important to perform scaling on them


In [9]:
# implementing the minmax scaler by using python only
def minmax_scaler(df):
    df = df.copy()
    for column in df.columns:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df

In [39]:
X = df.drop('price', axis = 1)
y = pd.DataFrame(df['price'].values.reshape(df.shape[0], 1))

In [40]:
scaled_X = minmax_scaler(X)
scaled_y = minmax_scaler(y)

We should convert target column (i.e. price) into one dimensional array. It has become 2D due to scaling that we did above but now we should change to 1D

In [41]:
scaled_y_array = np.array(scaled_y)
scaled_y_array.reshape(20, )

array([0.05237037, 0.65185185, 0.22222222, 0.31851852, 0.14074074,
       0.04444444, 0.76296296, 0.91111111, 0.13333333, 1.        ,
       0.37037037, 0.8       , 0.04444444, 0.05925926, 0.51111111,
       0.07407407, 0.11851852, 0.20740741, 0.51851852, 0.        ])

## $Price = W1 * X1 (age) + W2 * X2 (bedrooms) + bias$

### 1. Batch Gradient Descent Implementation